In [101]:
# Suburb growth data Victoria
import pandas as pd
import locale
from locale import atof

growth_data_to_use = pd.read_csv("Resources/suburb_growth_data.csv")
growth_data_to_use.head()

,Year,GCCSA,SA4,SA3,SA2,Area Type,Area Name,Total,Males \n0-4,Males \n5-9,...,Females \n40-44,Females \n45-49,Females \n50-54,Females \n55-59,Females \n60-64,Females \n65-69,Females \n70-74,Females \n75-79,Females \n80-84,Females \n85 and over
0,2016,2RVIC,201,20101,201011001,SA2,Alfredton,"11,852",454,500,...,439,438,411,340,290,259,179,132,71,80
1,2016,2RVIC,201,20101,201011002,SA2,Ballarat,"12,301",357,330,...,410,437,451,457,363,369,294,248,182,255
2,2016,2RVIC,201,20101,201011003,SA2,Ballarat - North,"23,539",798,784,...,764,788,836,761,737,674,477,415,333,418
3,2016,2RVIC,201,20101,201011004,SA2,Ballarat - South,"24,654",889,784,...,716,732,744,756,734,682,512,414,317,366
4,2016,2RVIC,201,20101,201011005,SA2,Buninyong,"7,311",243,286,...,275,243,233,258,227,207,128,89,58,30


In [22]:
# Select only columns we think (at this moment) we will need, the SA columns are 
# references to other definitions of areas so could get complivcated if we can't match Area Names with other datafiles
# GCCSA is 2 categories, Greater Melbourne and Regional Victoria
# TEAM please fel free to add columns you think you may need

growth_data_to_use = growth_data_to_use[["Year", "Area Name", "Total", "GCCSA", "SA4", "SA3","SA2", "Area Type"]]
growth_data_to_use.dtypes   

Year          int64
Area Name    object
Total        object
GCCSA        object
SA4           int64
SA3           int64
SA2           int64
Area Type    object
dtype: object

In [134]:
# Create lists by year if needed for further calculations
growth_data_by_year = growth_data_to_use.copy()
years = [2016,2021,2026,2031,2036]
year_totals =[]
for x in years:
    year_totals.append(growth_data_by_year.loc[growth_data_by_year["Year"]==x, ["Year","Area Name","GCCSA", "Total"]])

data_2016 = pd.DataFrame(year_totals[0])
data_2021 = pd.DataFrame(year_totals[1])
data_2026 = pd.DataFrame(year_totals[2])
data_2031 = pd.DataFrame(year_totals[3])
data_2036 = pd.DataFrame(year_totals[4])

GCCSA_df = data_2016[["Area Name", "GCCSA"]]

data_2016 = data_2016[["Area Name", "Total"]]
data_2016.rename(columns={"Total": "Total 2016"}, inplace=True)
data_2021 = data_2021[["Area Name","Total"]]
data_2021.rename(columns={"Total": "Total 2021"}, inplace=True)
data_2026 = data_2026[["Area Name", "Total"]]
data_2026.rename(columns={"Total": "Total 2026"}, inplace=True)
data_2031 = data_2031[["Area Name","Total"]]
data_2031.rename(columns={"Total": "Total 2031"}, inplace=True)
data_2036 = data_2036[["Area Name","Total"]]
data_2036.rename(columns={"Total": "Total 2036"}, inplace=True)

data_by_year = pd.merge(data_2016, data_2021, on = "Area Name", how = "left")
data_by_year = pd.merge(data_by_year, data_2026, on = "Area Name", how = "left")
data_by_year = pd.merge(data_by_year, data_2031, on = "Area Name", how = "left")
data_by_year = pd.merge(data_by_year, data_2036, on = "Area Name", how = "left")
data_by_year = pd.merge(data_by_year, GCCSA_df, on = "Area Name", how = "left")

data_by_year["Total 2016"]= data_by_year["Total 2016"].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)
data_by_year["Total 2021"]= data_by_year["Total 2021"].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)
data_by_year["Total 2026"]= data_by_year["Total 2026"].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)
data_by_year["Total 2031"]= data_by_year["Total 2031"].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)
data_by_year["Total 2036"]= data_by_year["Total 2036"].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)

# data_by_year.dtypes
data_by_year["Total Growth"] = data_by_year["Total 2036"] - data_by_year["Total 2016"]
data_by_year["Total Growth %"] = data_by_year["Total Growth"] / data_by_year["Total 2016"]
data_by_year.dtypes

Area Name          object
Total 2016        float64
Total 2021        float64
Total 2026        float64
Total 2031        float64
Total 2036        float64
GCCSA              object
Total Growth      float64
Total Growth %    float64
dtype: object

In [151]:
minimum_growth = 5000
minumum_growth_perc = 0.5

top_growth_areas = data_by_year.loc[(data_by_year["Total Growth"]>minimum_growth) & (data_by_year["Total Growth %"]>minumum_growth_perc)]
top_growth_areas

,Area Name,Total 2016,Total 2021,Total 2026,Total 2031,Total 2036,GCCSA,Total Growth,Total Growth %
0,Alfredton,11852.0,15513.0,19227.0,22932.0,26437.0,2RVIC,14585.0,1.230594
5,Delacombe,7195.0,9489.0,12714.0,15973.0,19196.0,2RVIC,12001.0,1.667964
23,Strathfieldsaye,8775.0,10613.0,12826.0,15132.0,17437.0,2RVIC,8662.0,0.987123
24,White Hills - Ascot,11914.0,14934.0,17644.0,20184.0,22487.0,2RVIC,10573.0,0.887443
33,Bannockburn,6549.0,8023.0,9684.0,11390.0,13093.0,2RVIC,6544.0,0.999237
...,...,...,...,...,...,...,...,...,...
390,Werribee - South,11776.0,17116.0,20836.0,24655.0,28856.0,2GMEL,17080.0,1.450408
391,Wyndham Vale,24270.0,33524.0,44369.0,55077.0,65947.0,2GMEL,41677.0,1.717223
392,Point Cook - East,12818.0,19601.0,21479.0,22269.0,23083.0,2GMEL,10265.0,0.800827
394,Point Cook - South,15147.0,20394.0,23098.0,26693.0,30303.0,2GMEL,15156.0,1.000594
